<a href="https://colab.research.google.com/github/rprimi/IR_bm25/blob/main/proj0_is_bm25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Building a Simple Information Retrieval System using BM25

## Description

Exercício para seleção aluno especial - Curso "Deep Learning aplicado a sistemas de buscas", FEEC-Unicamp.

**Objectives:** 
- Create basics of Information Retrieval (IR) andusing the BM25 ranking algorithm in Python.

**Deliverables:**
- A github repository
- Google Colab notebooks and associated functions and libraries if any, with the code for the IR system
- A brief report describing the implementation details, results, how to test the IR system and how chatGPT helped you with the project.
- Evaluation of the model in the CISI collection
http://ir.dcs.gla.ac.uk/resources/test_collections/cisi/

**Grading:**  
The project will be evaluated based on the following criteria:
- Completion of the project deliverables (partial is OK to submit)
- Implementation accuracy
- Quality of the report
- Code readability and commenting
- How chatGPT was used




## Learning about BM25
Prompts for Bing ChatGPT4: 

Claro, eu posso explicar em detalhes o que é cada elemento da fórmula acima score(D,Q) usando fórmulas em LaTeX.

A fórmula score(D,Q) calcula a relevância de um documento D para uma consulta Q, usando uma função de ranqueamento baseada no modelo probabilístico²³. Ela é composta pelos seguintes elementos:

- $\mathrm{IDF}(q_i)$ é o inverso da frequência do documento do termo $q_i$, que é uma medida de quão informativo ou discriminativo é o termo para diferenciar os documentos relevantes dos não relevantes. Ele é calculado como:


$$
\mathrm{IDF}(q_i) = \log \frac{N - n(q_i) + 0.5}{n(q_i) + 0.5}
$$


onde N é o número total de documentos na coleção e $n(q_i)$ é o número de documentos que contêm o termo $q_i$. O IDF aumenta à medida que o termo se torna mais raro na coleção, e diminui à medida que se torna mais comum.

- $f(q_i,D)$ é a frequência do termo $q_i$ no documento D, que é uma medida de quão importante ou relevante é o termo para o conteúdo do documento. Ela pode ser simplesmente a contagem do número de vezes que o termo aparece no documento, ou alguma outra função que leve em conta a posição, a proximidade ou a ponderação dos termos.

- $k_1$ e b são parâmetros ajustáveis que controlam a influência da frequência do termo e do comprimento do documento na pontuação final. Eles podem variar dependendo da coleção, da consulta ou da tarefa de recuperação. Valores típicos são $k_1 \in [1.2, 2]$ e $b = 0.75$².

- $(k_1 + 1)$ e $(k_1 \cdot (1 - b + b \cdot |D| / \mathrm{avgdl}))$ são fatores de normalização que evitam que documentos muito longos ou muito curtos tenham vantagem ou desvantagem na pontuação final. Eles levam em conta o comprimento do documento D ($|D|$), comparando-o com o comprimento médio dos documentos na coleção ($\mathrm{avgdl}$). O parâmetro b controla o grau de normalização: quanto mais próximo de 1, mais sensível ao comprimento; quanto mais próximo de 0, menos sensível.

A fórmula score(D,Q) soma os produtos dos elementos acima para cada termo da consulta Q, dando assim uma pontuação global para o documento D em relação à consulta Q. Quanto maior a pontuação, maior a relevância estimada.

Espero que isso tenha esclarecido os elementos da fórmula BM25.

Origem: conversa com o Bing, 16/02/2023(1) (PDF) A Tutorial on the BM25F Model - ResearchGate. https://www.researchgate.net/publication/308991534_A_Tutorial_on_the_BM25F_Model Acessado 16/02/2023.
(2) Okapi BM25 - Wikipedia. https://en.wikipedia.org/wiki/Okapi_BM25 Acessado 16/02/2023.
(3) Okapi BM25 - formulasearchengine. https://www.formulasearchengine.com/wiki/Okapi_BM25 Acessado 16/02/2023.
(4) Practical BM25 - Part 2: The BM25 Algorithm and its Variables. https://www.elastic.co/blog/practical-bm25-part-2-the-bm25-algorithm-and-its-variables Acessado 16/02/2023.

1. Aprendendo sobre BM25 com o chatGPT do new Bing

Olá, este é o Bing. Eu posso te ajudar. 😊

BM25 é uma função de classificação usada por mecanismos de busca para estimar a relevância de documentos para uma determinada consulta de pesquisa¹². É baseada no modelo probabilístico de recuperação de informação desenvolvido nas décadas de 1970 e 1980 por Stephen E. Robertson, Karen Spärck Jones e outros².

BM25 é uma função que pontua um conjunto de documentos com base nos termos da consulta que aparecem em cada documento, independentemente da proximidade entre os termos dentro do documento². É uma família de funções de pontuação com componentes e parâmetros ligeiramente diferentes. Uma das instâncias mais proeminentes da função é a seguinte²:

$$
\mathrm{score}(D,Q) = \sum_{i=1}^{n} \mathrm{IDF}(q_i) \cdot \frac{f(q_i,D) \cdot (k_1 + 1)}{f(q_i,D) + k_1 \cdot (1 - b + b \cdot |D| / \mathrm{avgdl})}
$$


onde:

- `D` é um documento
- `Q` é uma consulta
- `f(q_i,D)` é a frequência do termo `q_i` no documento `D`
- `|D|` é o comprimento do documento `D` em palavras
- `avgdl` é o comprimento médio dos documentos na coleção
- `k_1` e `b` são parâmetros livres, geralmente escolhidos como `k_1 = 2,0` e `b = 0,75`
- `IDF(q_i)` é o peso do termo inverso da frequência do documento para o termo `q_i`, calculado como:

```
IDF(q_i) = log((N - n(q_i) + 0.5) / (n(q_i) + 0.5))
```

onde:

- `N` é o número total de documentos na coleção
- `n(q_i)` é o número de documentos que contêm o termo `q_i`

A ideia por trás do BM25 é que quanto mais vezes um termo da consulta aparece em um documento, maior será a relevância desse documento para a consulta. No entanto, essa relevância também depende da frequência do termo na coleção inteira (quanto mais raro for o termo, maior será seu peso), do comprimento do documento (documentos mais curtos tendem a ter pontuações mais altas) e dos parâmetros livres que podem ser ajustados para diferentes cenários.

Existem várias implementações em Python da função BM25 disponíveis online⁶⁷⁹ [^10^] ¹¹. Uma delas é a biblioteca python-bm25⁶, que pode ser instalada usando pip:


Origem: conversa com o Bing, 16/02/2023(1) Okapi BM25 - Wikipedia. https://en.wikipedia.org/wiki/Okapi_BM25 Acessado 16/02/2023.
(2) Okapi BM25 - Wikipedia. https://en.wikipedia.org/wiki/Okapi_BM25 Acessado 16/02/2023.
(3) python-bm25 | Python implementation of BM25 function | Search Engine .... https://kandi.openweaver.com/python/fanta-mnix/python-bm25#:~:text=python-bm25%20is%20a%20Python%20library%20typically%20used%20in,Python%20implementation%20of%20BM25%20function%20for%20document%20retrieval Acessado 16/02/2023.
(4) BM25 using Python Gensim Package | Search Engine. https://iamgeekydude.com/2022/12/25/bm25-using-python-gensim-package-search-engine-nlp/ Acessado 16/02/2023.
(5) GitHub - nhirakawa/BM25: A Python implementation of the BM25 ranking .... https://github.com/nhirakawa/BM25 Acessado 16/02/2023.
(6) GitHub - fanta-mnix/python-bm25: Python implementation of BM25 function .... https://github.com/fanta-mnix/python-bm25 Acessado 16/02/2023.
(7) GitHub - xianchen2/Text_Retrieval_BM25: Python implementation of the .... https://github.com/xianchen2/Text_Retrieval_BM25 Acessado 16/02/2023.
(8) Machine Learning Approach for Improved BM25 Retrieval. https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/LearningBM25MSRTechReport.pdf Acessado 16/02/2023.
(9) Information retrieval evaluation of precision, recall, f-score, AP, MAP .... https://stackoverflow.com/questions/40457331/information-retrieval-evaluation-of-precision-recall-f-score-ap-map-in-pytho Acessado 16/02/2023.
(10) Information Retrieval with document Re-ranking with BERT and BM25. https://medium.com/@papai143/information-retrieval-with-document-re-ranking-with-bert-and-bm25-7c29d738df73 Acessado 16/02/2023.
(11) python-bm25 | Python implementation of BM25 function | Search Engine .... https://kandi.openweaver.com/python/fanta-mnix/python-bm25 Acessado 16/02/2023.

In [ ]:

```
pip install python-bm25
```

Um exemplo simples de uso dessa biblioteca é o seguinte:

```python
# Importar a classe BM25Okapi
from bm25 import BM25Okapi

# Criar uma lista de documentos como listas de palavras
docs = [
    ["black", "cat", "white", "cat"],
    ["cat", "outer", "space"],
    ["wag", "dog"]
]

# Criar um objeto BM25Okapi com os documentos
bm25 = BM25Okapi(docs)

# Criar uma consulta como lista de palavras
query = ["cat"]

# Calcular as pontuações dos documentos para a consulta usando get_scores()
scores = bm25.get_scores(query)

# Imprimir as pontuações
print(scores)
```

A saída deste código será:

```
[2.8327298855090213, 2.8327298855090213, 0]
```
#  Isso significa que os dois primeiros documentos têm a mesma pontuação BM25 para a consulta "cat" e são mais relevantes do que o terceiro documento.

# Espero que isso tenha esclarecido o que é BM25 e como implementá-lo em Python.

Claro, eu posso explicar cada elemento da fórmula BM25.

A fórmula BM25 é a seguinte²:

$$
score(D,Q) = sum_{i=1}^{n} IDF(q_i) * (f(q_i,D) * (k_1 + 1)) / (f(q_i,D) + k_1 * (1 - b + b * |D| / avgdl))
$$


Os elementos da fórmula são:

- `score(D,Q)` é a pontuação BM25 do documento `D` para a consulta `Q`
- `sum_{i=1}^{n}` é a soma sobre todos os termos da consulta `q_i` que aparecem no documento `D`
- `IDF(q_i)` é o peso do termo inverso da frequência do documento para o termo `q_i`, que mede o quão informativo é o termo para a consulta. É calculado como:

$$IDF(q_i) = log((N - n(q_i) + 0.5) / (n(q_i) + 0.5))$$


onde:

- $N$ é o número total de documentos na coleção
- `n(q_i)` é o número de documentos que contêm o termo `q_i`

Quanto menor for o valor de `n(q_i)`, maior será o valor de `IDF(q_i)` e maior será a contribuição do termo para a pontuação final.

- `(f(q_i,D) * (k_1 + 1)) / (f(q_i,D) + k_1 * (1 - b + b * |D| / avgdl))` é um fator de normalização que leva em conta a frequência do termo no documento, o comprimento do documento e os parâmetros livres. É composto por:

  - `f(q_i,D)` é a frequência do termo `q_i` no documento `D`, ou seja, quantas vezes ele aparece no texto. Quanto maior for esse valor, maior será a relevância do documento para o termo.
  - `(k_1 + 1)` e `(k_1 * (1 - b + b * |D| / avgdl))` são dois componentes que ajustam a influência da frequência do termo na pontuação final. Eles dependem dos parâmetros livres:
    - `k_1` controla o quanto a pontuação aumenta com base na frequência do termo. Valores maiores de `k_1` significam que as diferenças na frequência do termo têm mais impacto na pontuação, enquanto valores menores significam que as diferenças têm menos impacto. O valor padrão geralmente usado é `k_1 = 2,0`.
    - `b` controla o quanto a pontuação é afetada pelo comprimento do documento em relação ao comprimento médio dos documentos na coleção. Valores maiores de `b` favorecem documentos mais curtos, enquanto valores menores favorecem documentos mais longos. O valor padrão geralmente usado é `b = 0,75`.
    - |D|é o comprimento do documento D em palavras
    - avgdlé o comprimento médio dos documentos na coleção

Esses elementos juntos formam uma função de classificação que tenta estimar quão relevante é um documento para uma consulta com base nos termos que eles compartilham.

Espero que isso tenha esclarecido os elementos da fórmula BM25.

Origem: conversa com o Bing, 16/02/2023(1) Okapi BM25 - Wikipedia. https://en.wikipedia.org/wiki/Okapi_BM25 Acessado 16/02/2023.
(2) Practical BM25 - Part 2: The BM25 Algorithm and its Variables. https://www.elastic.co/blog/practical-bm25-part-2-the-bm25-algorithm-and-its-variables Acessado 16/02/2023.
(3) Document similarities with BM25 algorithm - MATLAB bm25Similarity. https://www.mathworks.com/help/textanalytics/ref/bm25similarity.html Acessado 16/02/2023.

In [ ]:
Para criar um código Python que implemente uma busca de informação usando o BM25 e avalie o sistema na coleção CISI, você pode seguir os seguintes passos:

1. Instale a biblioteca gensim, que fornece uma implementação do BM25 em Python¹. Você pode usar o comando `pip install gensim` no terminal.
2. Baixe os arquivos da coleção CISI do site http://ir.dcs.gla.ac.uk/resources/test_collections/cisi/. Você vai precisar dos arquivos CISI.ALL, CISI.QRY e CISI.REL, que contêm os documentos, as consultas e as relevâncias, respectivamente.
3. Crie uma função para ler os arquivos da coleção CISI e transformá-los em listas de strings. Por exemplo:

```
def read_cisi_file(filename):
  # Abre o arquivo no modo leitura
  with open(filename, "r") as f:
    # Lê todo o conteúdo do arquivo
    content = f.read()
    # Divide o conteúdo em registros separados pelo caractere ".I"
    records = content.split(".I")
    # Remove o primeiro registro vazio
    records.pop(0)
    # Cria uma lista vazia para armazenar os textos dos registros
    texts = []
    # Para cada registro na lista de registros
    for record in records:
      # Divide o registro em linhas
      lines = record.split("\n")
      # Remove a primeira linha que contém o número do registro
      lines.pop(0)
      # Junta as linhas restantes em um único texto
      text = " ".join(lines)
      # Adiciona o texto à lista de textos
      texts.append(text)
  # Retorna a lista de textos
  return texts

```

4. Use a função criada para ler os documentos, as consultas e as relevâncias da coleção CISI e armazená-los em variáveis. Por exemplo:

```
# Lê os documentos da coleção CISI e armazena na variável docs
docs = read_cisi_file("CISI.ALL")
# Lê as consultas da coleção CISI e armazena na variável queries
queries = read_cisi_file("CISI.QRY")
# Lê as relevâncias da coleção CISI e armazena na variável relevances
relevances = read_cisi_file("CISI.REL")

```

5. Crie uma função para pré-processar os textos dos documentos e das consultas, removendo pontuação, caracteres especiais, números, stopwords e aplicando stemização. Você pode usar a biblioteca nltk para isso. Por exemplo:

```
import re
import nltk

# Define uma lista de stopwords em inglês
stopwords = nltk.corpus.stopwords.words("english")
# Define um stemizador baseado no algoritmo de Porter
stemmer = nltk.stem.PorterStemmer()

def preprocess(text):
  # Converte o texto para minúsculas
  text = text.lower()
  # Remove pontuação, caracteres especiais e números usando expressões regulares
  text = re.sub(r"[^\w\s]", "", text)
  text = re.sub(r"\d+", "", text)
  # Divide o texto em tokens (palavras) usando espaços em branco como separador
  tokens = text.split()
  # Remove os tokens que são stopwords usando list comprehension 
  tokens = [token for token in tokens if token not in stopwords]
  # Aplica stemização aos tokens usando list comprehension 
  tokens = [stemmer.stem(token) for token in tokens]
  # Retorna a lista de tokens pré-processados 
  return tokens

```

6. Use a função criada para pré-processar os documentos e as consultas da coleção CISI e armazená-los em novas variáveis. Por exemplo:

```
# Pré-processa os documentos da coleção CISI e armazena na variável docs_tokens 
docs_tokens = [preprocess(doc) for doc in docs]
# Pré-processa as consultas da coleção CISI e armazena na variável queries_tokens 
queries_tokens = [preprocess(query) for query in queries]

```

7. Importe a classe BM25 do módulo gensim.summarization.bm25¹ e use-a para criar um objeto BM25 com base nos documentos pré-processados. Por exemplo:

```
from gensim.summarization.b

Origem: conversa com o Bing, 16/02/2023(1) How to use gensim BM25 ranking in python - Stack Overflow. https://stackoverflow.com/questions/40966014/how-to-use-gensim-bm25-ranking-in-python Acessado 16/02/2023.
(2) GitHub - nhirakawa/BM25: A Python implementation of the BM25 ranking .... https://github.com/nhirakawa/BM25 Acessado 16/02/2023.
(3) BM25 | Build your Own NLP Based Search Engine Using BM25 - Analytics Vidhya. https://www.analyticsvidhya.com/blog/2021/05/build-your-own-nlp-based-search-engine-using-bm25/ Acessado 16/02/2023.

In [ ]:
Here is a Python code that implements a information search retrieval using BM25 and evaluates the system on CISI collection. I have added comments to explain the details.

```python
# Importing libraries
import os
import re
import math
from gensim import corpora, models, similarities

# Defining a function to read and tokenize documents from a file
def read_docs(file):
    docs = []
    with open(file, "r") as f:
        for line in f:
            # Each document starts with .I followed by an id number
            if line.startswith(".I"):
                doc = ""
                doc_id = line.strip()
            # Each document ends with .X followed by references
            elif line.startswith(".X"):
                docs.append((doc_id, doc.split()))
            else:
                # Removing punctuation and numbers from the text
                line = re.sub("[^a-zA-Z\s]", "", line)
                doc += line.lower()
    return docs

# Defining a function to read and tokenize queries from a file
def read_queries(file):
    queries = []
    with open(file, "r") as f:
        for line in f:
            # Each query starts with .I followed by an id number
            if line.startswith(".I"):
                query = ""
                query_id = line.strip()
            # Each query ends with .W followed by text
            elif line.startswith(".W"):
                query += f.readline().lower()
                queries.append((query_id, query.split()))
    return queries

# Defining a function to read relevance judgments from a file
def read_relevance(file):
    relevance = {}
    with open(file, "r") as f:
        for line in f:
            # Each line contains a query id, a document id and a relevance score (1 or 0)
            qid, did, rel = line.split()
            # Storing the relevance judgments in a dictionary of sets
            if qid not in relevance:
                relevance[qid] = set()
            if rel == "1":
                relevance[qid].add(did)
    return relevance

# Reading and tokenizing the documents from cisi.all file
docs = read_docs("cisi.all")

# Creating a dictionary of terms from the documents
dictionary = corpora.Dictionary(doc[1] for doc in docs)

# Creating a corpus of bag-of-words vectors from the documents
corpus = [dictionary.doc2bow(doc[1]) for doc in docs]

# Initializing a BM25 model from gensim library using the corpus and dictionary 
bm25_model = models.Bm25Model(corpus=corpus, id2word=dictionary)

# Reading and tokenizing the queries from cisi.que file 
queries = read_queries("cisi.que")

# Reading the relevance judgments from cisi.rel file 
relevance = read_relevance("cisi.rel")

# Defining some evaluation metrics 
precision_at_10 = 0 # Precision at rank 10 
average_precision = 0 # Mean average precision 
recall_at_50 = 0 # Recall at rank 50 
num_queries = len(queries) # Number of queries 

# Looping through each query 
for query in queries:
    # Getting the query id and terms 
    qid, qterms = query 
    # Converting the query terms to bag-of-words vector 
    qvec = dictionary.doc2bow(qterms) 
    # Getting the BM25 scores for each document in the corpus using the query vector 
    scores = bm25_model.get_scores(qvec) 
    # Sorting the scores in descending order and getting the top 50 documents ids  
    ranked_docs_ids= sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:50]  
    ranked_docs_ids= [docs[i][0] for i in ranked_docs_ids]  
   
   ### Evaluating precision at rank 10 ###
   relevant_at_10= set(ranked_docs_ids[:10]) & relevance[qid]  
   precision_at_10+= len(relevant_at_10) / 10  

   ### Evaluating mean average precision ###
   num_relevant= 0  
   sum_precision= 0  
   for i, didin enumerate(ranked_docs_ids):  
       if didinrelevance[qid]:  
           num_relevant+= 1  
           sum_precision+= num_relevant / (i + 1)  
   average_precision+= sum_precision / len(relevance[qid])  

   ### Evaluating recall at rank 50 ###
   relevant_at_50= set(ranked_docs_ids) & relevance[qid]

Origem: conversa com o Bing, 16/02/2023(1) How to use gensim BM25 ranking in python - Stack Overflow. https://stackoverflow.com/questions/40966014/how-to-use-gensim-bm25-ranking-in-python Acessado 16/02/2023.
(2) BM25 | Build your Own NLP Based Search Engine Using BM25 - Analytics Vidhya. https://www.analyticsvidhya.com/blog/2021/05/build-your-own-nlp-based-search-engine-using-bm25/ Acessado 16/02/2023.
(3) svm - Implementation of Okapi BM25 in python - Stack Overflow. https://stackoverflow.com/questions/61877065/implementation-of-okapi-bm25-in-python Acessado 16/02/2023.
(4) Glasgow IDOM - CISI collection. http://ir.dcs.gla.ac.uk/resources/test_collections/cisi/ Acessado 16/02/2023.
(5) Average precisions for CISI collection: 1000 documents are added in 100 .... https://www.researchgate.net/figure/Average-precisions-for-CISI-collection-1000-documents-are-added-in-100-groups-of-10_fig2_228961662 Acessado 16/02/2023.
(6) Comparison of normalized error for the CISI collection: 1000 documents .... https://www.researchgate.net/figure/Comparison-of-normalized-error-for-the-CISI-collection-1000-documents-are-added-in-100_fig7_228961662 Acessado 16/02/2023.
(7) How to use gensim BM25 ranking in python - Stack Overflow. https://stackoverflow.com/questions/40966014/how-to-use-gensim-bm25-ranking-in-python Acessado 16/02/2023.
(8) rank-bm25 · PyPI. https://pypi.org/project/rank-bm25/ Acessado 16/02/2023.
(9) BM25 | Build your Own NLP Based Search Engine Using BM25 - Analytics Vidhya. https://www.analyticsvidhya.com/blog/2021/05/build-your-own-nlp-based-search-engine-using-bm25/ Acessado 16/02/2023.

BM25 is a ranking function that estimates the relevance of documents to a given search query based on the frequency and importance of the query terms in the documents². It is a bag-of-words retrieval function, which means it does not consider the order or proximity of the query terms within the document². It is based on the probabilistic retrieval framework developed by Stephen E. Robertson, Karen Spärck Jones, and others²⁵.

Origem: conversa com o Bing, 16/02/2023(1) Okapi BM25 - Wikipedia. https://en.wikipedia.org/wiki/Okapi_BM25 Acessado 16/02/2023.
(2) How does BM25 work? - Quora. https://www.quora.com/How-does-BM25-work Acessado 16/02/2023.
(3) BM25 | Build your Own NLP Based Search Engine Using BM25 - Analytics Vidhya. https://www.analyticsvidhya.com/blog/2021/05/build-your-own-nlp-based-search-engine-using-bm25/#:~:text=BM25%20is%20a%20simple%20Python%20package%20and%20can,occurrences%20of%20the%20search%20term%20in%20our%20tweet Acessado 16/02/2023.
(4) BM25 | Build your Own NLP Based Search Engine Using BM25 - Analytics Vidhya. https://www.analyticsvidhya.com/blog/2021/05/build-your-own-nlp-based-search-engine-using-bm25/ Acessado 16/02/2023.
(5) Practical BM25 - Part 2: The BM25 Algorithm and its Variables. https://www.elastic.co/blog/practical-bm25-part-2-the-bm25-algorithm-and-its-variables Acessado 16/02/2023.

In [ ]:
A information search using BM25 works as follows:

- First, you need to have a collection of documents that you want to search through. These documents can be anything from web pages, tweets, news articles, etc. You also need to have a query that represents what you are looking for.
- Second, you need to preprocess both the documents and the query by tokenizing them into words, removing stopwords and punctuation, stemming or lemmatizing them, etc. This will help reduce the noise and variability in the text data.
- Third, you need to create a dictionary of terms from the documents and assign each term an id. You also need to create a corpus of bag-of-words vectors from the documents, where each vector represents how many times each term appears in each document.
- Fourth, you need to initialize a BM25 model using the corpus and dictionary. The BM25 model will calculate some statistics for each term and document, such as term frequency (TF), inverse document frequency (IDF), document length (DL), average document length (AVDL), etc. These statistics will be used to compute the BM25 score for each document given a query.
- Fifth, you need to convert your query into a bag-of-words vector using the same dictionary. Then, you need to use the BM25 model to get the scores for each document in the corpus using the query vector. The scores are based on a formula that combines TF, IDF, DL, AVDL and some parameters that control how much weight is given to each factor. The higher the score, the more relevant the document is to the query.
- Sixth, you need to sort the scores in descending order and get a ranked list of documents that match your query. You can also limit the number of documents returned or apply some filters or thresholds based on your needs.

Here is a simple python code that illustrates these steps using gensim library:

```python
# Importing gensim library
from gensim import corpora, models

# Defining some sample documents
docs = ["The sky is blue.", "The sun is bright.", "The grass is green.", "The snow is white."]

# Defining a sample query
query = "What color is snow?"

# Preprocessing both documents and query by tokenizing them into words
docs = [doc.lower().split() for doc in docs]
query = query.lower().split()

# Creating a dictionary of terms from documents
dictionary = corpora.Dictionary(docs)

# Creating a corpus of bag-of-words vectors from documents
corpus = [dictionary.doc2bow(doc) for doc in docs]

# Initializing a BM25 model using corpus and dictionary
bm25_model = models.Bm25Model(corpus=corpus,id2word=dictionary)

# Converting query into bag-of-words vector using dictionary
query_vec = dictionary.doc2bow(query)

# Getting BM25 scores for each document in corpus using query vector
scores = bm25_model.get_scores(query_vec)

# Sorting scores in descending order and getting ranked list of documents ids 
ranked_docs_ids = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)

# Printing ranked list of documents with their scores 
for i in ranked_docs_ids:
    print(f"Document {i}: {docs[i]} - Score: {scores[i]}")
```

Origem: conversa com o Bing, 17/02/2023(1) nlp - Using BM25 to rank words - Data Science Stack Exchange. https://datascience.stackexchange.com/questions/75839/using-bm25-to-rank-words#:~:text=BM25%20is%20usually%20used%20in%20information%20retrieval.%20In,provided%20from%20the%20most%20relevant%20to%20the%20least. Acessado 17/02/2023.
(2) BM25 | Build your Own NLP Based Search Engine Using BM25 - Analytics Vidhya. https://www.analyticsvidhya.com/blog/2021/05/build-your-own-nlp-based-search-engine-using-bm25/ Acessado 17/02/2023.
(3) Okapi BM25 - Wikipedia. https://en.wikipedia.org/wiki/Okapi_BM25 Acessado 17/02/2023.
(4) BM25 using Python Gensim Package | Search Engine. https://iamgeekydude.com/2022/12/25/bm25-using-python-gensim-package-search-engine-nlp/ Acessado 17/02/2023.
(5) Practical BM25 - Part 2: The BM25 Algorithm and its Variables. https://www.elastic.co/blog/practical-bm25-part-2-the-bm25-algorithm-and-its-variables Acessado 17/02/2023.

To modify the code to work with any collection, you need to do the following steps:

- First, you need to load your collection of documents from a file or a database. You can use any format that suits your needs, such as JSON, CSV, XML, etc. You can also use any library that helps you read and parse your data, such as pandas, json, xmltodict, etc.
- Second, you need to preprocess your documents as before by tokenizing them into words, removing stopwords and punctuation, stemming or lemmatizing them, etc. You can use any library that helps you with these tasks, such as nltk, spacy, gensim.utils.simple_preprocess(), etc.
- Third, you need to create a dictionary of terms from your documents using gensim.corpora.Dictionary(). This will assign each term an id and store some statistics about the term frequency and document frequency.
- Fourth, you need to create a corpus of bag-of-words vectors from your documents using dictionary.doc2bow(). This will create a list of tuples for each document where each tuple contains a term id and its count in the document.
- Fifth, you need to initialize a BM25 model using gensim.summarization.bm25.BM25Model(). This will take the corpus and dictionary as input and calculate some statistics for each term and document that will be used for scoring.
- Sixth, you need to convert your query into a bag-of-words vector using dictionary.doc2bow() as before. Then, you need to use bm25_model.get_scores() to get the scores for each document in the corpus using the query vector. The scores are based on a formula that combines TF,IDF,Dl,AIDL and some parameters that control how much weight is given to each factor. The higher the score,the more relevantthe document is tothe query.
-Seventh,youneedtosortthescoresindescendingorderandgetarankedlistofdocumentsidsusing sorted().Youcanalsolimitthenumberofdocumentsreturnedorapplysomefiltersorthresholdsbasedonyourneeds.

Here is an example of modifying the code to work with any collection:

```python
# Importing libraries
import json
from gensim import corpora
from gensim.summarization import bm25

# Loading collection of documents from JSON file
with open("docs.json", "r") as f:
    docs = json.load(f)

# Preprocessing documents by tokenizing them into words
docs = [doc.lower().split() for doc in docs]

# Creating a dictionary of terms from documents
dictionary = corpora.Dictionary(docs)

# Creating a corpus of bag-of-words vectors from documents
corpus = [dictionary.doc2bow(doc) for doc in docs]

# Initializing a BM25 model using corpus and dictionary
bm25_model = bm25.BM25Model(corpus=corpus,id2word=dictionary)

# Defining a sample query
query = "What color is snow?"

# Converting query into bag-of-words vector using dictionary
query_vec = dictionary.doc2bow(query.split())

# Getting BM25 scores for each document in corpus using query vector
scores = bm25_model.get_scores(query_vec)

# Sorting scores in descending order and getting ranked list of documents ids 
ranked_docs_ids = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)

# Printing ranked list of documents with their scores 
for i in ranked_docs_ids:
    print(f"Document {i}: {docs[i]} - Score: {scores[i]}")
```

Origem: conversa com o Bing, 17/02/2023(1) Configure relevance scoring - Azure Cognitive Search | Microsoft Learn. https://learn.microsoft.com/en-us/azure/search/index-ranking-similarity Acessado 17/02/2023.
(2) How to use gensim BM25 ranking in python - Stack Overflow. https://stackoverflow.com/questions/40966014/how-to-use-gensim-bm25-ranking-in-python Acessado 17/02/2023.
(3) Practical BM25 - Part 2: The BM25 Algorithm and its Variables. https://www.elastic.co/blog/practical-bm25-part-2-the-bm25-algorithm-and-its-variables Acessado 17/02/2023.